In [1]:
!pip install datasets evaluate transformers[sentencepiece] evaluate openpyxl seqeval optuna

In [2]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 26.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import os
import re

import evaluate
import nltk
import numpy as np
import optuna
import pandas as pd
import spacy
import torch
from datasets import Dataset, DatasetDict
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    pipeline,
)

Подгрузка данных

In [4]:
df = pd.read_excel('выгрузка_артикулов.xlsx')

In [5]:
df.head()

,text,entity
0,"Тест-система для идентификации линий ГМО ""Соя ...",GM-202-50
1,Адаптер/муфта F603462/0,F603462/0
2,БОЛТ KG00468961,KG00468961
3,V ремень ksn1000 Holdijk Haamberg (106384),106384
4,Поршень гидроцилиндра № 0387304,0387304


Разметка токенов

In [6]:
# Зафиксируем random state для воспроизводимости
RANDOM_STATE = 0

In [7]:
# Загрузка spaCy модели для русского языка (small)
nlp = spacy.load('ru_core_news_sm')

In [8]:
def annotate_text_with_bert(text, entity):
    """
    Аннотирует текст, выделяя в нем указанную сущность (entity)
        с использованием токенизации SpaCy.

    Args:
        text: Текст, в котором нужно найти и выделить сущность.
        entity: Сущность, которую необходимо найти и пометить в тексте.

    Returns:
        Кортеж, содержащий два списка:
            - tokens: Список токенов текста.
            - labels: Список меток для каждого токена.  O означает вне сущности,
              B-ENTITY - начало сущности, I-ENTITY - продолжение сущности.
    """
    doc = nlp(text)

    # Токенизация текста с использованием SpaCy
    tokens = [token.text for token in doc]
    labels = ['O'] * len(tokens)  # Изначально все токены помечаются как O

    # Токенизация сущности с использованием SpaCy
    entity_doc = nlp(entity)
    entity_tokens = [token.text for token in entity_doc]
    entity_len = len(entity_tokens)

    # Поиск всех вхождений сущности в тексте (с учетом регистра и пробелов)
    for i in range(len(tokens) - entity_len + 1):
        if tokens[i:i+entity_len] == entity_tokens:
            labels[i] = 'B-ENTITY'
            for j in range(1, entity_len):
                labels[i+j] = 'I-ENTITY'

    return tokens, labels

In [9]:
# Применение функции к каждому тексту
df[['tokens', 'labels']] = df.apply(lambda row: pd.Series(annotate_text_with_bert(row['text'], row['entity'])), axis=1)

In [10]:
df.head()

,text,entity,tokens,labels
0,"Тест-система для идентификации линий ГМО ""Соя ...",GM-202-50,"[Тест, -, система, для, идентификации, линий, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,Адаптер/муфта F603462/0,F603462/0,"[Адаптер, /, муфта, F603462/0]","[O, O, O, B-ENTITY]"
2,БОЛТ KG00468961,KG00468961,"[БОЛТ, KG00468961]","[O, B-ENTITY]"
3,V ремень ksn1000 Holdijk Haamberg (106384),106384,"[V, ремень, ksn1000, Holdijk, Haamberg, (, 106...","[O, O, O, O, O, O, B-ENTITY, O]"
4,Поршень гидроцилиндра № 0387304,0387304,"[Поршень, гидроцилиндра, №, 0387304]","[O, O, O, B-ENTITY]"


In [11]:
label_names = ['O', 'B-ENTITY', 'I-ENTITY']

label2id = {label: i for i, label in enumerate(label_names)}

# Переворачиваем словарь
id2label = {value: key for key, value in label2id.items()}

In [12]:
def convert_labels_to_ids(labels_list, labels_to_id):
    return [label2id[label] for label in labels_list]

In [13]:
# Кодируем сущности в id
df['ner_tags'] = df['labels'].apply(lambda x: convert_labels_to_ids(x, label2id))

In [14]:
df.head()

,text,entity,tokens,labels,ner_tags
0,"Тест-система для идентификации линий ГМО ""Соя ...",GM-202-50,"[Тест, -, система, для, идентификации, линий, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Адаптер/муфта F603462/0,F603462/0,"[Адаптер, /, муфта, F603462/0]","[O, O, O, B-ENTITY]","[0, 0, 0, 1]"
2,БОЛТ KG00468961,KG00468961,"[БОЛТ, KG00468961]","[O, B-ENTITY]","[0, 1]"
3,V ремень ksn1000 Holdijk Haamberg (106384),106384,"[V, ремень, ksn1000, Holdijk, Haamberg, (, 106...","[O, O, O, O, O, O, B-ENTITY, O]","[0, 0, 0, 0, 0, 0, 1, 0]"
4,Поршень гидроцилиндра № 0387304,0387304,"[Поршень, гидроцилиндра, №, 0387304]","[O, O, O, B-ENTITY]","[0, 0, 0, 1]"


Подготовка данных

In [65]:
train_df, test_df = train_test_split(df[['text', 'tokens', 'entity', 'ner_tags']], random_state=RANDOM_STATE, shuffle=True, test_size=0.2)

In [16]:
train_df, val_df = train_test_split(train_df, random_state=RANDOM_STATE, shuffle=True, test_size=0.1)

In [17]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

In [18]:
full_dataset = DatasetDict({'train': train_dataset, 'test': test_dataset, 'val': val_dataset})

In [19]:
def align_labels_with_tokens(labels, word_ids):
    """
    Выравнивает набор меток (labels) в соответствии со списком идентификаторов слов (word_ids).

    Это необходимо, потому что после добавления специальных токенов ([CLS] и [SEP])
    и разделения некоторых слов список токенов становится длиннее исходного.

    Args:
        labels: Список исходных меток для каждого слова.
        word_ids: Список идентификаторов слов, сопоставляющих каждый токен
            с соответствующим словом в исходном тексте. `None` означает, что токен
            является специальным токеном (например, [CLS], [SEP]).

    Returns:
        list: Новый список меток, выровненный с токенами.  Метка `-100` используется
            для специальных токенов, чтобы они игнорировались при вычислении потерь.
    """

    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            """
            Проверяем, изменился ли идентификатор текущего слова.
            Если `word_id` отличается от `current_word`, значит, мы перешли к новому слову.
            """
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [20]:
def tokenizer_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'], truncation=True, is_split_into_words=True
        )
    all_labels = examples['ner_tags']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

In [39]:
def prepare_data_with_tokenizer(tokenizer, dataset):
    def tokenizer_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples['tokens'], truncation=True, is_split_into_words=True
        )
        all_labels = examples['ner_tags']
        new_labels = []
        for i, labels in enumerate(all_labels):
            word_ids = tokenized_inputs.word_ids(i)
            new_labels.append(align_labels_with_tokens(labels, word_ids))

        tokenized_inputs['labels'] = new_labels
        return tokenized_inputs

    # Применение токенизатора к данным
    tokenized_dataset = dataset.map(
        tokenizer_and_align_labels,
        batched=True,
        remove_columns=dataset['train'].column_names,
    )
    return tokenized_dataset

In [22]:
# Метрика для NER
metric = evaluate.load('seqeval')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [23]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l!= -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p,l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        'precision': all_metrics['overall_precision'],
        'recall': all_metrics['overall_recall'],
        'f1': all_metrics['overall_f1'],
        'accuracy': all_metrics['overall_accuracy']
    }

Обучение моделей

In [24]:
def test_model(model_checkpoint, dataset):
    """
    Обучает модель для задачи токенизации и сохраняет ее, чтобы ее можно было загрузить по имени.

    Args:
        model_checkpoint (str): Имя или путь к предобученной модели (например, "cointegrated/rubert-tiny2").
        dataset (DatasetDict): Объект DatasetDict, содержащий наборы данных 'train', 'val' и 'test'.
        model_name (str, optional): Имя, под которым будет сохранена обученная модель.
                                     Директория для сохранения будет создана как 'bert-finetuned-ner/{model_name}'.
                                     По умолчанию "bert-finetuned-ner".

    Returns:
        Trainer: Объект Trainer после обучения.
    """

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    label_ids = prepare_data_with_tokenizer(tokenizer, dataset)

    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        id2label=id2label,
        label2id=label2id,
    )

    output_dir = os.path.join("bert-finetuned-ner", model_checkpoint)
    os.makedirs(output_dir, exist_ok=True)

    args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        report_to='none',
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        warmup_ratio=0.1,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=label_ids['train'],
        eval_dataset=label_ids['val'],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    trainer.train()
    test_results = trainer.evaluate(eval_dataset=label_ids['test'])

    print('Modelname:', model_checkpoint)
    print(test_results)
    print()

    trainer.save_model() #  Сохраняет и модель и токенизатор

    print(f"Модель сохранена в директорию: {output_dir}")

    return trainer

In [25]:
model_checkpoints = [
    'cointegrated/rubert-tiny2', # Супер популярная легкая модель берта на русском
    'ai-forever/sbert_large_nlu_ru' # Популярная тяжелая модель BERT на русском от Сбера
]

In [26]:
for model_checkpoint in model_checkpoints:
    test_model(model_checkpoint, full_dataset)


Map:   0%|          | 0/7372 [00:00<?, ? examples/s]

Map:   0%|          | 0/2048 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-24-1f3dc90b22d4>:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.159800,0.123278,0.782656,0.892365,0.833918,0.954999
2,0.110500,0.115145,0.800875,0.916145,0.854641,0.960330
3,0.094800,0.098292,0.837900,0.918648,0.876418,0.968984


...

Map:   0%|          | 0/2048 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-24-1f3dc90b22d4>:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.079100,0.048299,0.953545,0.976220,0.964750,0.989845
2,0.037800,0.043391,0.965728,0.987484,0.976485,0.992037
3,0.019400,0.048384,0.969136,0.982478,0.975761,0.991379


Modelname: ai-forever/sbert_large_nlu_ru
{'eval_loss': 0.07822465896606445, 'eval_precision': 0.9547488811536549, 'eval_recall': 0.967741935483871, 'eval_f1': 0.9612015018773468, 'eval_accuracy': 0.9862638978763457, 'eval_runtime': 18.2023, 'eval_samples_per_second': 112.513, 'eval_steps_per_second': 14.064, 'epoch': 3.0}

Модель сохранена в директорию: bert-finetuned-ner/ai-forever/sbert_large_nlu_ru


Тюнинг гиперпараметров

Из-за ограниченных вычислительных мощностей тюнинг не делаем, но его можно было бы сделать так


In [27]:
# model_checkpoint = 'ai-forever/sbert_large_nlu_ru'

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# label_ids = prepare_data_with_tokenizer(tokenizer, full_dataset)

# data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# model = AutoModelForTokenClassification.from_pretrained(
#     model_checkpoint,
#     id2label=id2label,
#     label2id=label2id,
# )

In [28]:
# # Определяем функцию объектива для Optuna
# def objective(trial):
#     learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-4)
#     num_train_epochs = trial.suggest_int('num_train_epochs', 2, 5)
#     weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
#     per_device_train_batch_size = trial.suggest_categorical(
#         'per_device_train_batch_size',
#          [2, 4, 8]
#     )

#     # Обновляем аргументы тренировки'
#     training_args = TrainingArguments(
#         output_dir='./results',
#         evaluation_strategy='epoch',
#         save_strategy='epoch',
#         learning_rate=learning_rate,
#         num_train_epochs=num_train_epochs,
#         weight_decay=weight_decay,
#         per_device_train_batch_size=per_device_train_batch_size,
#         report_to='none'
#     )

#     # Создаем тренер с новыми гиперпараметрами
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=label_ids['train'],
#         eval_dataset=label_ids['val'],
#         data_collator=data_collator,
#         compute_metrics=compute_metrics,
#         tokenizer=tokenizer
#     )

#     # Обучаем модель и получаем результаты
#     trainer.train()
#     eval_results = trainer.evaluate()

#     # Возвращаем F1-score как целевую метрику для Optuna
#     return eval_results['eval_f1']

# # Создаем исследование Optuna
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=5)

# # Выводим лучшие найденные гиперпараметры
# print('Best trial:')
# trial = study.best_trial
# print('  Value: {}'.format(trial.value))
# print('  Params: ')
# for key, value in trial.params.items():
#     print(f'    {key}: {value}')


Подгружаем модель

In [32]:
model_name = 'ai-forever/sbert_large_nlu_ru'  # Или имя, которое вы использовали при обучении
model_path = os.path.join('bert-finetuned-ner', model_name)

In [73]:
pipeline_ner = pipeline(
    task='ner',
    model=model_path,
    batch_size=8,
    tokenizer=model_path,
    device='cuda',
)

Device set to use cuda


In [74]:
def extract_articles_from_pipeline(ner_results):
    """
    Извлекает артикулы (сущности типа ENTITY) из результатов NER-пайплайна,
    конкатенируя их в одну строку.

    Args:
        ner_results: Список словарей, где каждый словарь представляет результат
                    NER-пайплайна для одного токена и содержит ключи 'word' (токен)
                    и 'entity' (метка).

    Returns:
        Строка, содержащая конкатенированные артикулы, очищенные от символа #,
        или пустая строка, если артикулы не найдены.
    """
    output = ''  # Инициализируем пустую строку для результата
    current_article = '' # Инициализируем пустую строку для текущего артикула

    for result in ner_results:
        token = result['word']
        label = result['entity']

        if label == 'B-ENTITY':
            if current_article:
                output += current_article  # Конкатенируем предыдущий артикул
            current_article = token  # Начинаем новый артикул
        elif label == 'I-ENTITY':
            current_article += token  # Добавляем токен к текущему артикулу
        else:
            if current_article:
                output += current_article  # Конкатенируем текущий артикул
                current_article = '' # Очищаем current_article

    # Добавляем последний артикул, если он есть
    if current_article:
        output += current_article
    if output:
        output = re.sub('#', '', output)  # Удаляем лишний символ
    return output

In [174]:
# Возьмем случайное описание из теста и сделаем прогноз
sample = test_df.sample(1)
index = sample.index[0]

text = sample.loc[index, 'text']
entity = sample.loc[index, 'entity']

ner_results = pipeline_ner(text)

# Извлечение артикулов
prediction = extract_articles_from_pipeline(ner_results)
print('Описание:', text)
print('Сущность:', entity)
print('Извлеченный артикул:', prediction.upper())
print(ner_results)

Описание: Фильтр воздушный 261971
Сущность: 261971
Извлеченный артикул: 261971
[{'entity': 'B-ENTITY', 'score': np.float32(0.9999044), 'index': 5, 'word': '261', 'start': 17, 'end': 20}, {'entity': 'I-ENTITY', 'score': np.float32(0.9998884), 'index': 6, 'word': '##97', 'start': 20, 'end': 22}, {'entity': 'I-ENTITY', 'score': np.float32(0.9998907), 'index': 7, 'word': '##1', 'start': 22, 'end': 23}]


Единственный минус модели, что она uncased, то есть если в артикулах встретятся буквы, то она их сделает строчными. Так как во всех артикулах буквы заглавные, то мы просто сделаем все артикулы заглавными.

In [175]:
# Сделаем прогноз на всем тесте
predictions = []

for text in test_df['text']:
    ner_results = pipeline_ner(text)
    prediction = extract_articles_from_pipeline(ner_results)
    predictions.append(prediction.upper())

test_df['predictions'] = predictions

In [176]:
test_df.to_csv('test_bert_ner.csv')